<a href="https://colab.research.google.com/github/DomTHK/DLO_Lab/blob/main/Abgabe/DLO_Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
# https://drive.google.com/drive/folders/1hNFbb-Ksc9Qdyx0_CB7tTJFy7-UNwYwy?usp=sharing 

# Bildgröße 300x200 RGB

from google.colab import drive
import torchvision 
from torchvision import transforms

from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torchvision import datasets, transforms



In [ ]:
drive.mount("/content/drive", force_remount=True)

In [68]:
# Config Feld
path = f"/content/drive/MyDrive/Colab Notebooks/data/*/*/*.png"

device = "cuda" if torch.cuda.is_available() else "cpu"

In [50]:
all_files = glob(path, recursive = True)
print(len(all_files) )
assert len(all_files) == 2188, "Der Datensatz ist nicht gleich 2188"

2188


In [62]:
transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])

dataset = datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/data', transform=transform)

In [66]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [70]:
@torch.no_grad()
def accuracy(x, y, model):
    with torch.no_grad():
        prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

In [71]:
@torch.no_grad()
def loss(x, y, model):
    prediction = model(x)
    loss = loss_fn(prediction, y)
    return loss.item()

In [69]:
def get_model():
    model = nn.Sequential(
        nn.Linear(28 * 28, 200),  # 28 * 28 Input-Nodes | 40 Output-Nodes
        nn.Sigmoid(),  # Aktivierungsfunktion
        nn.Linear(200, 40),
        nn.Sigmoid(),
        nn.Dropout(p=0.2),
        nn.Linear(40, 40),
        nn.Sigmoid(),  # Aktivierungsfunktion
        nn.Linear(40, 20),
        nn.Sigmoid(),
        nn.Linear(20, 10)  # 20 Input-Nodes | 10 Output-Nodes
    ).to(device)
    loss_fn = nn.CrossEntropyLoss()  # Kreuzentropie
    optimizer = SGD(model.parameters(), lr=1e-1)
    return model, loss_fn, optimizer

In [67]:
for images, labels in dataloader:
    pass

tensor([[[[0.3059, 0.3059, 0.3137,  ..., 0.3020, 0.3059, 0.3098],
          [0.2902, 0.2980, 0.3059,  ..., 0.2980, 0.3020, 0.3098],
          [0.2980, 0.2980, 0.2980,  ..., 0.2980, 0.2941, 0.2980],
          ...,
          [0.2863, 0.2863, 0.2863,  ..., 0.2706, 0.2667, 0.2667],
          [0.2902, 0.2863, 0.2863,  ..., 0.2784, 0.2706, 0.2745],
          [0.2941, 0.2902, 0.2863,  ..., 0.2863, 0.2784, 0.2824]],

         [[0.6980, 0.7020, 0.7098,  ..., 0.6980, 0.7020, 0.7098],
          [0.6941, 0.7098, 0.7176,  ..., 0.7020, 0.7059, 0.7137],
          [0.7020, 0.7059, 0.7098,  ..., 0.7020, 0.6980, 0.7020],
          ...,
          [0.7059, 0.7020, 0.7059,  ..., 0.6980, 0.6941, 0.6941],
          [0.7098, 0.7059, 0.7059,  ..., 0.7059, 0.6980, 0.6980],
          [0.7098, 0.7059, 0.7059,  ..., 0.7137, 0.7020, 0.7059]],

         [[0.2941, 0.2980, 0.3059,  ..., 0.2941, 0.2980, 0.3020],
          [0.2824, 0.2980, 0.3020,  ..., 0.2902, 0.2941, 0.3020],
          [0.2902, 0.2941, 0.2941,  ..., 0

KeyboardInterrupt: ignored